# Read modified station XML files with extra metadata in Json format 

## Fei Zhang

2020-07-14

In [1]:
from obspy import read_inventory

our_new_station_xml ="OA.CF28_station_inv_modified_json.xml"
our_inv = read_inventory(our_new_station_xml,format='STATIONXML')
# print(our_inv.networks[0].stations[0].extra)

extra_meta = our_inv.networks[0].stations[0].extra.gpsclockcorrection.value

In [2]:
print(type(extra_meta))

<class 'str'>


In [3]:
print(extra_meta)


{
    "network_code":"OA",
    "station_code":"CF28",

    "orient_correction": {
        "start_dt": "2017-11-07T09:07:34.930000Z",
        "end_dt":   "2018-08-23T03:52:29.528000Z",
        "azimuth_correction": -5.0
    },

  "gps_clock_corrections": [
    {
      "date": "2018-01-04",
      "seconds": -1.3375814425470127
    },
    {
      "date": "2018-01-05",
      "seconds": -1.110449564656099
    },
    {
      "date": "2018-01-06",
      "seconds": -0.9032476255118933
    }
  ]

}



In [4]:
import json

mdata = json.loads(extra_meta)

print (mdata)
print(mdata.keys())
print(type(mdata['gps_clock_corrections']))

for corr in mdata['gps_clock_corrections']:
    print (corr["date"], corr["seconds"] )

{'network_code': 'OA', 'station_code': 'CF28', 'orient_correction': {'start_dt': '2017-11-07T09:07:34.930000Z', 'end_dt': '2018-08-23T03:52:29.528000Z', 'azimuth_correction': -5.0}, 'gps_clock_corrections': [{'date': '2018-01-04', 'seconds': -1.3375814425470127}, {'date': '2018-01-05', 'seconds': -1.110449564656099}, {'date': '2018-01-06', 'seconds': -0.9032476255118933}]}
dict_keys(['network_code', 'station_code', 'orient_correction', 'gps_clock_corrections'])
<class 'list'>
2018-01-04 -1.3375814425470127
2018-01-05 -1.110449564656099
2018-01-06 -0.9032476255118933


In [6]:

print (mdata['gps_clock_corrections'])

[{'date': '2018-01-04', 'seconds': -1.3375814425470127}, {'date': '2018-01-05', 'seconds': -1.110449564656099}, {'date': '2018-01-06', 'seconds': -0.9032476255118933}]


In [7]:
# https://pbpython.com/pandas-list-dict.html

import pandas as pd

df_clock_correction = pd.DataFrame(mdata['gps_clock_corrections'])
df_clock_correction.head()

,date,seconds
0,2018-01-04,-1.337581
1,2018-01-05,-1.110450
2,2018-01-06,-0.903248


In [8]:
print (df_clock_correction.tail())

         date   seconds
0  2018-01-04 -1.337581
1  2018-01-05 -1.110450
2  2018-01-06 -0.903248


In [9]:
# get correction for a certain date YYYY-MM-DD
print(df_clock_correction.loc[df_clock_correction["date"] =="2018-01-04"])

         date   seconds
0  2018-01-04 -1.337581


In [10]:
type(df_clock_correction.query('date == "2018-01-04" ').seconds)

pandas.core.series.Series

In [11]:
df_clock_correction["date"] =="2018-01-04"

0     True
1    False
2    False
Name: date, dtype: bool

In [12]:
df_clock_correction.query('date == "2018-01-04" ').seconds.values[0]

-1.3375814425470127

In [13]:
df_clock_correction.describe()


,seconds
count,3.000000
mean,-1.117093
std,0.217243
min,-1.337581
25%,-1.224016
50%,-1.110450
75%,-1.006849
max,-0.903248


In [14]:
df_clock_correction.dtypes

date        object
seconds    float64
dtype: object

In [18]:
# What if there are more than one Station Node, such as AU.HTT in testdata/network_AU_0.xml, and OA.CF28 
stations =our_inv.networks[0].stations
print (len(stations))

3


# Dealing with multiple station-nodes in a network.station inventory file

See Jira: https://gajira.atlassian.net/browse/PV-130

 What if there are more than one Station Node, such as AU.HTT in testdata/network_AU_0.xml. 
 And our_new_station_xml ="OA.CF28_station_inv_modified_json.xml"
 
The multple station code represent the same station with different channel configuraiton over different time periods.
 

In [15]:

# create the modified xml file by running
# python  add_time_corrections.py  /g/data/ha3/Passive/SHARED_DATA/GPS_Clock/corrections/AU.HTT_clock_correction.csv ../../tests/testdata/network_AU_0.xml 

def inspect_stations( our_new_station_xml ):
    
    our_inv = read_inventory(our_new_station_xml,format='STATIONXML')

    #csv_str = our_inv.networks[0].stations[0].extra.gpsclockcorrection.value

    stations =our_inv.networks[0].stations
    
    if len(stations)>=2:
        print (our_new_station_xml)
        print ("Warning: more than one Station Node = %s" %len(stations))
    
    return stations


In [16]:
#our_new_station_xml ="/g/data/ha3/Passive/SHARED_DATA/GPS_Clock/StationXML_with_time_corrections2/AU.HTT_station_inv_modified.xml"
inspect_stations(our_new_station_xml)

OA.CF28_station_inv_modified_json.xml


[Station CF28 (CF28)
	Station Code: CF28
	Channel Count: None/None (Selected/Total)
	2017-11-06T02:02:50.000000Z - 2018-06-09T06:06:39.000000Z
	Access: None 
	Latitude: -21.50, Longitude: 139.50, Elevation: 381.7 m
	Available Channels:
		CF28.0M.HHZ, CF28.0M.HHN, CF28.0M.HHE,
 Station CF28 (CF28)
	Station Code: CF28
	Channel Count: None/None (Selected/Total)
	2018-06-09T06:21:41.000000Z - 2018-08-12T05:41:01.000000Z
	Access: None 
	Latitude: -21.50, Longitude: 139.50, Elevation: 381.7 m
	Available Channels:
		CF28.0M.HHZ, CF28.0M.HHN, CF28.0M.HHE,
 Station CF28 (CF28)
	Station Code: CF28
	Channel Count: None/None (Selected/Total)
	2018-08-12T05:45:10.000000Z - 2018-08-24T10:02:20.000000Z
	Access: None 
	Latitude: -21.50, Longitude: 139.50, Elevation: 381.7 m
	Available Channels:
		CF28.0M.HHZ, CF28.0M.HHN, CF28.0M.HHE]

In [17]:

import os, glob

xmlfolder = "/g/data/ha3/Passive/SHARED_DATA/GPS_Clock/StationXML_with_time_corrections2/"

for axmlfile in glob.glob(os.path.join(xmlfolder,"*.xml")):
    
    stations = inspect_stations(axmlfile)
    
#     if len(stations)>=2:
#         for astation in stations:
#             print("### ", astation.code, astation.extra)

/g/data/ha3/Passive/SHARED_DATA/GPS_Clock/StationXML_with_time_corrections2/OA.BY22_station_inv_modified.xml
/g/data/ha3/Passive/SHARED_DATA/GPS_Clock/StationXML_with_time_corrections2/AU.HTT_station_inv_modified.xml
/g/data/ha3/Passive/SHARED_DATA/GPS_Clock/StationXML_with_time_corrections2/AU.MUN_station_inv_modified.xml
/g/data/ha3/Passive/SHARED_DATA/GPS_Clock/StationXML_with_time_corrections2/OA.CE28_station_inv_modified.xml
/g/data/ha3/Passive/SHARED_DATA/GPS_Clock/StationXML_with_time_corrections2/OA.CE22_station_inv_modified.xml
/g/data/ha3/Passive/SHARED_DATA/GPS_Clock/StationXML_with_time_corrections2/OA.CF28_station_inv_modified.xml
